![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [19]:
data = pd.read_csv("rental_info.csv")

In [20]:
data["rental_length"] = pd.to_datetime(data["return_date"]) - pd.to_datetime(data["rental_date"])
data["rental_length_days"] = data["rental_length"].dt.days

data["deleted_scenes"] = data["special_features"].str.contains("Deleted Scenes").astype(int)
data["behind_the_scenes"] = data["special_features"].str.contains("Behind the Scenes").astype(int)

In [21]:
print("Columns before dropping:", data.columns)

drop_columns = ["special_features", "rental_length", "rental_date", "return_date"]

X = data.drop(columns=drop_columns + ["rental_length_days"])
y = data["rental_length_days"]

print("Shape of X after dropping:", X.shape)

Columns before dropping: Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'rental_length',
       'rental_length_days', 'deleted_scenes', 'behind_the_scenes'],
      dtype='object')
Shape of X after dropping: (15861, 14)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [23]:
lasso_model = Lasso(alpha=0.3, random_state=9)
lasso_model.fit(X_train, y_train)

selected_features = lasso_model.coef_ > 0
X_train_selected = X_train.loc[:, selected_features]
X_test_selected = X_test.loc[:, selected_features]

In [24]:

linear_model = LinearRegression()
linear_model.fit(X_train_selected, y_train)
y_pred_linear = linear_model.predict(X_test_selected)
mse_linear = mean_squared_error(y_test, y_pred_linear)


param_grid = {
    'n_estimators': np.arange(1, 101),
    'max_depth': np.arange(1, 11)
}

In [25]:

rf_model = RandomForestRegressor(random_state=9)
random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid, cv=5, random_state=9)
random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=9),
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])},
                   random_state=9)

In [26]:
best_params = random_search.best_params_
final_rf_model = RandomForestRegressor(n_estimators=best_params["n_estimators"],
                                       max_depth=best_params["max_depth"],
                                       random_state=9)
final_rf_model.fit(X_train, y_train)
y_pred_rf = final_rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)

In [27]:
if mse_rf < mse_linear:
    best_model, best_mse = final_rf_model, mse_rf
else:
    best_model, best_mse = linear_model, mse_linear

print(f"Best Model MSE: {best_mse:.2f}")


Best Model MSE: 2.23
